In [1]:
import pandas as pd
import sqlite3
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score, mean_poisson_deviance, mean_gamma_deviance
from sklearn.metrics import mean_squared_error, mean_absolute_error, classification_report
import xgboost as xgb

conn = sqlite3.connect(r'C:\Users\Owner\dev\algobetting\infra\data\db\algobetting.db')

df = pd.read_sql_query("""
                        SELECT DISTINCT
                            f.*,
                            ms.summary_shots as shots,
                            ms.opp_summary_shots as opp_shots
                        FROM 
                            team_all_features_365_005_no_div_changes f
                        JOIN
                            fbref_match_summary_v2 ms
                                ON ms.match_url = f.match_url
                                AND ms.team = f.team
                        WHERE 
                            f.team_rolling_summary_minutes IS NOT NULL
                       """, conn)

df

,match_url,match_date,season,division,team,opp_team,is_home,gw,is_promoted?,opp_is_promoted?,...,opp_team_rolling_conceded_defense_challenges_lost,opp_team_rolling_conceded_defense_blocked_shots,opp_team_rolling_conceded_defense_blocked_passes,opp_team_rolling_conceded_defense_tackles_interceptions,opp_team_rolling_conceded_defense_clearances,opp_team_rolling_conceded_defense_errors,opp_team_rolling_conceded_keeper_psxg,opp_team_rolling_conceded_is_promoted?,shots,opp_shots
0,https://fbref.com/en/matches/e4bb1c35/Tottenha...,2025-05-25 00:00:00,2024-2025,Premier League,Tottenham,Brighton,1,38,0,0,...,8.796946,3.393454,7.020603,26.403010,27.177737,0.862971,1.299603,0.169180,3.0,22.0
1,https://fbref.com/en/matches/e4bb1c35/Tottenha...,2025-05-25 00:00:00,2024-2025,Premier League,Brighton,Tottenham,0,38,0,0,...,7.949406,3.378971,9.378588,32.038948,23.558919,0.911136,1.805191,0.162674,22.0,3.0
2,https://fbref.com/en/matches/1ff370e8/Bournemo...,2025-05-25 00:00:00,2024-2025,Premier League,Bournemouth,Leicester City,1,38,0,1,...,6.606014,2.506894,6.904780,22.901258,21.697324,0.636969,1.811285,0.107786,20.0,3.0
3,https://fbref.com/en/matches/1ff370e8/Bournemo...,2025-05-25 00:00:00,2024-2025,Premier League,Leicester City,Bournemouth,0,38,1,0,...,7.520887,4.153870,6.746572,23.732322,26.770789,1.451179,1.460762,0.143648,3.0,20.0
4,https://fbref.com/en/matches/36844e73/Newcastl...,2025-05-25 00:00:00,2024-2025,Premier League,Newcastle Utd,Everton,1,38,0,0,...,6.895930,2.600829,6.075265,24.063503,26.142993,0.567784,1.251694,0.176309,17.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40271,https://fbref.com/en/matches/7a5b27d7/Palermo-...,2022-08-13 00:00:00,2022-2023,Serie B,Perugia,Palermo,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,11.0
40272,https://fbref.com/en/matches/e8e75a75/Como-Cag...,2022-08-13 00:00:00,2022-2023,Serie B,Como,Cagliari,1,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,12.0
40273,https://fbref.com/en/matches/99979e25/Cittadel...,2022-08-13 00:00:00,2022-2023,Serie B,Cittadella,Pisa,1,1,0,0,...,7.713670,2.909231,6.588752,27.403947,19.741195,0.237578,1.160446,0.140356,12.0,16.0
40274,https://fbref.com/en/matches/99979e25/Cittadel...,2022-08-13 00:00:00,2022-2023,Serie B,Pisa,Cittadella,0,1,0,0,...,5.815970,3.866474,9.173457,27.363966,20.635355,0.149176,0.957865,0.205776,16.0,12.0


In [2]:
df["under_11.5"] = np.where(df["shots"] < 11.5, 1, 0)

df

,match_url,match_date,season,division,team,opp_team,is_home,gw,is_promoted?,opp_is_promoted?,...,opp_team_rolling_conceded_defense_blocked_shots,opp_team_rolling_conceded_defense_blocked_passes,opp_team_rolling_conceded_defense_tackles_interceptions,opp_team_rolling_conceded_defense_clearances,opp_team_rolling_conceded_defense_errors,opp_team_rolling_conceded_keeper_psxg,opp_team_rolling_conceded_is_promoted?,shots,opp_shots,under_11.5
0,https://fbref.com/en/matches/e4bb1c35/Tottenha...,2025-05-25 00:00:00,2024-2025,Premier League,Tottenham,Brighton,1,38,0,0,...,3.393454,7.020603,26.403010,27.177737,0.862971,1.299603,0.169180,3.0,22.0,1
1,https://fbref.com/en/matches/e4bb1c35/Tottenha...,2025-05-25 00:00:00,2024-2025,Premier League,Brighton,Tottenham,0,38,0,0,...,3.378971,9.378588,32.038948,23.558919,0.911136,1.805191,0.162674,22.0,3.0,0
2,https://fbref.com/en/matches/1ff370e8/Bournemo...,2025-05-25 00:00:00,2024-2025,Premier League,Bournemouth,Leicester City,1,38,0,1,...,2.506894,6.904780,22.901258,21.697324,0.636969,1.811285,0.107786,20.0,3.0,0
3,https://fbref.com/en/matches/1ff370e8/Bournemo...,2025-05-25 00:00:00,2024-2025,Premier League,Leicester City,Bournemouth,0,38,1,0,...,4.153870,6.746572,23.732322,26.770789,1.451179,1.460762,0.143648,3.0,20.0,1
4,https://fbref.com/en/matches/36844e73/Newcastl...,2025-05-25 00:00:00,2024-2025,Premier League,Newcastle Utd,Everton,1,38,0,0,...,2.600829,6.075265,24.063503,26.142993,0.567784,1.251694,0.176309,17.0,14.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40271,https://fbref.com/en/matches/7a5b27d7/Palermo-...,2022-08-13 00:00:00,2022-2023,Serie B,Perugia,Palermo,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,11.0,1
40272,https://fbref.com/en/matches/e8e75a75/Como-Cag...,2022-08-13 00:00:00,2022-2023,Serie B,Como,Cagliari,1,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,12.0,1
40273,https://fbref.com/en/matches/99979e25/Cittadel...,2022-08-13 00:00:00,2022-2023,Serie B,Cittadella,Pisa,1,1,0,0,...,2.909231,6.588752,27.403947,19.741195,0.237578,1.160446,0.140356,12.0,16.0,0
40274,https://fbref.com/en/matches/99979e25/Cittadel...,2022-08-13 00:00:00,2022-2023,Serie B,Pisa,Cittadella,0,1,0,0,...,3.866474,9.173457,27.363966,20.635355,0.149176,0.957865,0.205776,16.0,12.0,0


In [3]:
X = df.drop(columns=["team", "opp_team", "match_url", "match_date", "shots", "opp_shots", "under_11.5"])
# Use dummies for nominal categorical features
X = pd.get_dummies(X, columns=["season", "division"], drop_first=True)
y = df["under_11.5"]

# Use matches after a certain date as test
cutoff_date = '2024-08-01'  # adjust as needed
df['match_date'] = pd.to_datetime(df['match_date'])

train_mask = df['match_date'] < cutoff_date
test_mask = df['match_date'] >= cutoff_date

X_train = X[train_mask]
X_test = X[test_mask]
y_train = y[train_mask]
y_test = y[test_mask]  

In [4]:
# Train XGBoost classifier
model = xgb.XGBClassifier(enable_categorical=True, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.3f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Get feature importance (returns a dictionary)
feature_importance_dict = model.get_booster().get_score(importance_type='gain')

# Convert dictionary to DataFrame
importance_df = pd.DataFrame(
    list(feature_importance_dict.items()), 
    columns=["feature", "importance"]
).sort_values("importance", ascending=False)

print(importance_df.head(25))

Accuracy: 0.636

Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.67      0.65      2709
           1       0.63      0.60      0.62      2559

    accuracy                           0.64      5268
   macro avg       0.64      0.64      0.64      5268
weighted avg       0.64      0.64      0.64      5268

                                               feature  importance
259              opp_team_rolling_conceded_summary_sca  125.093880
0                                              is_home   73.102318
17                            team_rolling_summary_sca   54.217281
7                           team_rolling_summary_shots   52.086121
137       team_rolling_conceded_pass_types_passes_live   38.345985
249            opp_team_rolling_conceded_summary_shots   26.905102
22             team_rolling_summary_progressive_passes   19.177935
36            team_rolling_possession_carries_distance   18.334465
324                           

In [5]:
def create_team_opp_interactions(df, importance_df, min_importance=10):
   """
   Create interaction features between team_rolling and opp_team_rolling features
   that have importance above the threshold.
   """
   # Get features above importance threshold
   important_features = importance_df[importance_df['importance'] >= min_importance]['feature'].tolist()
   
   # Find team_rolling and opp_team_rolling features
   team_rolling_features = [f for f in important_features if f.startswith('team_rolling')]
   opp_team_rolling_features = [f for f in important_features if f.startswith('opp_team_rolling')]
   
   print(f"Team rolling features with importance > {min_importance}:")
   for f in team_rolling_features:
       print(f"  {f}")
   
   print(f"\nOpp team rolling features with importance > {min_importance}:")
   for f in opp_team_rolling_features:
       print(f"  {f}")
   
   # Create a copy of the dataframe to add new features
   df_enhanced = df.copy()
   
   # Track created interactions to avoid duplicates
   created_interactions = set()
   
   # Try different matching patterns
   for team_feat in team_rolling_features:
       potential_matches = []
       
       # Pattern 1: Direct match (team_rolling_X vs opp_team_rolling_X)
       team_stat = team_feat.replace('team_rolling_', '')
       opp_feat_direct = f'opp_team_rolling_{team_stat}'
       if opp_feat_direct in opp_team_rolling_features:
           potential_matches.append(opp_feat_direct)
       
       # Pattern 2: Conceded match (team_rolling_X vs opp_team_rolling_conceded_X)
       opp_feat_conceded = f'opp_team_rolling_conceded_{team_stat}'
       if opp_feat_conceded in opp_team_rolling_features:
           potential_matches.append(opp_feat_conceded)
       
       # Pattern 3: Team conceded vs opp regular (team_rolling_conceded_X vs opp_team_rolling_X)
       if team_feat.startswith('team_rolling_conceded_'):
           team_stat_no_conceded = team_feat.replace('team_rolling_conceded_', '')
           opp_feat_no_conceded = f'opp_team_rolling_{team_stat_no_conceded}'
           if opp_feat_no_conceded in opp_team_rolling_features:
               potential_matches.append(opp_feat_no_conceded)
       
       # Create interactions for all matches found
       for opp_feat in potential_matches:
           interaction_name = f'interaction_{team_feat}_x_{opp_feat}'
           
           if interaction_name not in created_interactions:
               df_enhanced[interaction_name] = df[team_feat] * df[opp_feat]
               created_interactions.add(interaction_name)
               print(f"Created: {interaction_name}")
   
   print(f"\nTotal interactions created: {len(created_interactions)}")
   return df_enhanced

# Create enhanced datasets
X_train_enhanced = create_team_opp_interactions(X_train, importance_df, min_importance=10)
X_test_enhanced = create_team_opp_interactions(X_test, importance_df, min_importance=10)

# Train enhanced model
model_enhanced = xgb.XGBClassifier(
        enable_categorical=True, 
        importance_type='gain',
        reg_alpha=1.0,  # L1 regularization
        reg_lambda=1.0,  # L2 regularization
        max_depth=4,    # Reduce complexity
        random_state=42
    )
model_enhanced.fit(X_train_enhanced, y_train)

# Make predictions with enhanced features
y_pred_enhanced = model_enhanced.predict(X_test_enhanced)

# Evaluate enhanced model
accuracy_enhanced = accuracy_score(y_test, y_pred_enhanced)
print(f"\nEnhanced Model Accuracy: {accuracy_enhanced:.3f}")
print(f"Original Model Accuracy: {accuracy:.3f}")
print(f"Improvement: {accuracy_enhanced - accuracy:.3f}")

print("\nEnhanced Model Classification Report:")
print(classification_report(y_test, y_pred_enhanced))

# Get feature importance for enhanced model
feature_importance_enhanced = model_enhanced.get_booster().get_score(importance_type='gain')
importance_df_enhanced = pd.DataFrame(
   list(feature_importance_enhanced.items()), 
   columns=["feature", "importance"]
).sort_values("importance", ascending=False)

print("\nTop 20 features in enhanced model:")
print(importance_df_enhanced.head(20))

Team rolling features with importance > 10:
  team_rolling_summary_sca
  team_rolling_summary_shots
  team_rolling_conceded_pass_types_passes_live
  team_rolling_summary_progressive_passes
  team_rolling_possession_carries_distance
  team_rolling_possession_touches_att_3rd
  team_rolling_possession_progressive_passes_received
  team_rolling_summary_shots_on_target
  team_rolling_conceded_summary_passes
  team_rolling_possession_passes_received
  team_rolling_conceded_keeper_psxg
  team_rolling_conceded_summary_gca
  team_rolling_summary_passes_pct
  team_rolling_possession_carries_into_penalty_area

Opp team rolling features with importance > 10:
  opp_team_rolling_conceded_summary_sca
  opp_team_rolling_conceded_summary_shots
  opp_team_rolling_conceded_possession_touches_att_3rd
  opp_team_rolling_is_promoted?
  opp_team_rolling_conceded_possession_touches_att_pen_area
  opp_team_rolling_possession_touches_att_3rd
  opp_team_rolling_conceded_summary_pens_att
  opp_team_rolling_conced

# Hyperparam Tuning

In [6]:
import xgboost as xgb
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import cross_val_score, StratifiedKFold
import pickle
import os
import optuna
from optuna.integration import XGBoostPruningCallback
import numpy as np

# Quick toggle - set to True to run search, False to load existing model
RUN_OPTUNA_SEARCH = True  # Change to False if you want to load existing model

model_file = 'best_xgb_model_optuna_enhanced.pkl'

def objective(trial):
    # Refined parameter search space based on your successful results
    params = {
        'enable_categorical': True,
        'random_state': 42,
        'tree_method': 'hist',
        'eval_metric': 'logloss',
        
        # Core tree parameters - tightened around your best values
        'max_depth': trial.suggest_int('max_depth', 3, 4),  # Your best was 3, explore 3-4
        'learning_rate': trial.suggest_float('learning_rate', 0.025, 0.045, log=True),  # Narrow around 0.031
        'n_estimators': trial.suggest_int('n_estimators', 300, 500),  # Your best was 377, explore higher
        
        # Regularization - refined around your optimal values
        'reg_alpha': trial.suggest_float('reg_alpha', 3.5, 6.5),  # Centered around your 4.75
        'reg_lambda': trial.suggest_float('reg_lambda', 5.0, 8.0),  # Centered around your 6.29
        
        # Feature sampling - your patterns show lower colsample_bytree works well
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.25, 0.45),  # Narrow around your 0.357
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.75, 0.90),  # Around your 0.804
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.65, 0.85),  # Around your 0.729
        
        # Data sampling - refined around your successful value
        'subsample': trial.suggest_float('subsample', 0.75, 0.90),  # Around your 0.807
        'min_child_weight': trial.suggest_int('min_child_weight', 4, 7),  # Around your 5
        
        # Additional parameters - refined ranges
        'gamma': trial.suggest_float('gamma', 0.5, 1.2),  # Around your 0.814
        'max_delta_step': trial.suggest_int('max_delta_step', 0, 2),  # Your best was 1
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 0.95, 1.05),  # Very close to 1.0
        
        # Additional exploration parameters
        'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),
        'max_leaves': trial.suggest_int('max_leaves', 0, 15) if trial.suggest_categorical('use_max_leaves', [True, False]) else 0,
    }
    
    # Remove max_leaves if grow_policy is depthwise (XGBoost requirement)
    if params['grow_policy'] == 'depthwise':
        params['max_leaves'] = 0
    
    # Create model with pruning callback for efficiency
    pruning_callback = XGBoostPruningCallback(trial, 'validation_0-logloss')
    
    model = xgb.XGBClassifier(**params)
    
    # Use stratified k-fold for more robust evaluation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(model, X_train_enhanced, y_train, cv=skf, scoring='accuracy', n_jobs=-1)
    
    return cv_scores.mean()

if RUN_OPTUNA_SEARCH or not os.path.exists(model_file):
    print("Running enhanced Optuna optimization with refined parameter ranges...")
    print("Based on your successful results - focusing on promising regions...")
    
    # Create study with more advanced sampler
    sampler = optuna.samplers.TPESampler(seed=42, n_startup_trials=30)  # More startup trials
    study = optuna.create_study(direction='maximize', sampler=sampler)
    
    # Add your current best trial as a starting point
    study.enqueue_trial({
        'max_depth': 3,
        'learning_rate': 0.031444802109958,
        'n_estimators': 377,
        'reg_alpha': 4.753385037999372,
        'reg_lambda': 6.287596648316482,
        'colsample_bytree': 0.3570361530008187,
        'colsample_bylevel': 0.804120600153049,
        'colsample_bynode': 0.7295049567365972,
        'subsample': 0.8074190410592694,
        'min_child_weight': 5,
        'gamma': 0.8144881355784646,
        'max_delta_step': 1,
        'scale_pos_weight': 1.012591945911166,
        'grow_policy': 'depthwise',
        'use_max_leaves': False
    })
    
    # Add some variations around your best parameters
    variations = [
        {'max_depth': 4, 'n_estimators': 400, 'learning_rate': 0.035},
        {'max_depth': 3, 'n_estimators': 450, 'learning_rate': 0.028},
        {'reg_alpha': 5.5, 'reg_lambda': 7.0, 'gamma': 0.9},
    ]
    
    for variation in variations:
        base_params = {
            'max_depth': 3,
            'learning_rate': 0.031444802109958,
            'n_estimators': 377,
            'reg_alpha': 4.753385037999372,
            'reg_lambda': 6.287596648316482,
            'colsample_bytree': 0.3570361530008187,
            'colsample_bylevel': 0.804120600153049,
            'colsample_bynode': 0.7295049567365972,
            'subsample': 0.8074190410592694,
            'min_child_weight': 5,
            'gamma': 0.8144881355784646,
            'max_delta_step': 1,
            'scale_pos_weight': 1.012591945911166,
            'grow_policy': 'depthwise',
            'use_max_leaves': False
        }
        base_params.update(variation)
        study.enqueue_trial(base_params)
    
    # Optimize with more trials and no timeout
    print("Running optimization without timeout - will complete all trials...")
    study.optimize(objective, n_trials=300)  # Increased trials, no timeout
    
    print(f"Best trial: {study.best_trial.number}")
    print(f"Best value: {study.best_value:.4f}")
    print(f"Best params: {study.best_params}")
    
    # Show improvement
    if study.best_value > 0.6699:
        improvement = (study.best_value - 0.6699) * 100
        print(f"Improvement over previous best: +{improvement:.2f} percentage points")
    
    # Train final model with best parameters
    best_params = study.best_params.copy()
    
    # Clean up the parameters
    if 'use_max_leaves' in best_params:
        del best_params['use_max_leaves']
    if best_params.get('grow_policy') == 'depthwise':
        best_params['max_leaves'] = 0
    
    best_params.update({'enable_categorical': True, 'random_state': 42, 'tree_method': 'hist'})
    
    model = xgb.XGBClassifier(**best_params)
    model.fit(X_train_enhanced, y_train)
    
    # Save model
    with open(model_file, 'wb') as f:
        pickle.dump({'model': model, 'study': study}, f)
    
    print(f"Model and study saved to {model_file}")
    
else:
    print(f"Loading existing model from {model_file}")
    with open(model_file, 'rb') as f:
        saved_data = pickle.load(f)
        model = saved_data['model']
        if 'study' in saved_data:
            study = saved_data['study']
            print(f"Loaded study with {len(study.trials)} trials")

# Make predictions
y_pred = model.predict(X_test_enhanced)
y_pred_proba = model.predict_proba(X_test_enhanced)

# Evaluate
accuracy = accuracy_score(y_test, y_pred)
print(f"\nFinal Model Accuracy: {accuracy:.4f}")

if accuracy >= 0.67:
    print("🎉 SUCCESS! Achieved ≥67% accuracy!")
else:
    print(f"Current accuracy: {accuracy:.4f}, target: ≥0.67")
    print(f"Gap to target: {(0.67 - accuracy)*100:.2f} percentage points")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Show model parameters
print(f"\nFinal model parameters:")
for param, value in model.get_params().items():
    if param in ['max_depth', 'learning_rate', 'n_estimators', 'reg_alpha', 'reg_lambda', 
                 'colsample_bytree', 'colsample_bylevel', 'colsample_bynode', 'subsample', 
                 'min_child_weight', 'gamma', 'max_delta_step', 'scale_pos_weight',
                 'grow_policy', 'max_leaves']:
        if isinstance(value, float):
            print(f"{param}: {value:.6f}")
        else:
            print(f"{param}: {value}")

# Feature importance analysis
if hasattr(model, 'feature_importances_'):
    print(f"\nTop 10 most important features:")
    if 'X_train_enhanced' in globals():
        feature_names = X_train_enhanced.columns if hasattr(X_train_enhanced, 'columns') else [f'feature_{i}' for i in range(len(model.feature_importances_))]
        importance_df = list(zip(feature_names, model.feature_importances_))
        importance_df.sort(key=lambda x: x[1], reverse=True)
        for i, (feature, importance) in enumerate(importance_df[:10]):
            print(f"{i+1:2d}. {feature}: {importance:.4f}")

# Additional insights if study is available
if 'study' in locals():
    print(f"\nOptimization insights:")
    print(f"Total trials: {len(study.trials)}")
    print(f"Best trial number: {study.best_trial.number}")
    
    # Show parameter importance
    try:
        importance = optuna.importance.get_param_importances(study)
        print(f"\nMost important hyperparameters:")
        for i, (param, imp) in enumerate(sorted(importance.items(), key=lambda x: x[1], reverse=True)[:8]):
            print(f"{i+1}. {param}: {imp:.4f}")
    except:
        pass
    
    # Show top trials
    print(f"\nTop 5 trials:")
    trials_df = [(trial.number, trial.value) for trial in study.trials if trial.value is not None]
    trials_df.sort(key=lambda x: x[1], reverse=True)
    for i, (trial_num, value) in enumerate(trials_df[:5]):
        print(f"{i+1}. Trial {trial_num}: {value:.4f}")

[I 2025-07-24 14:13:04,887] A new study created in memory with name: no-name-ea814f7e-63e0-47cd-ad31-4f4f8ea06c71


Running enhanced Optuna optimization with refined parameter ranges...
Based on your successful results - focusing on promising regions...
Running optimization without timeout - will complete all trials...


[I 2025-07-24 14:13:34,828] Trial 0 finished with value: 0.6698756504406009 and parameters: {'max_depth': 3, 'learning_rate': 0.031444802109958, 'n_estimators': 377, 'reg_alpha': 4.753385037999372, 'reg_lambda': 6.287596648316482, 'colsample_bytree': 0.3570361530008187, 'colsample_bylevel': 0.804120600153049, 'colsample_bynode': 0.7295049567365972, 'subsample': 0.8074190410592694, 'min_child_weight': 5, 'gamma': 0.8144881355784646, 'max_delta_step': 1, 'scale_pos_weight': 1.012591945911166, 'grow_policy': 'depthwise', 'use_max_leaves': False}. Best is trial 0 with value: 0.6698756504406009.
[I 2025-07-24 14:14:05,303] Trial 1 finished with value: 0.6666192788144151 and parameters: {'max_depth': 4, 'learning_rate': 0.035, 'n_estimators': 400, 'reg_alpha': 4.753385037999372, 'reg_lambda': 6.287596648316482, 'colsample_bytree': 0.3570361530008187, 'colsample_bylevel': 0.804120600153049, 'colsample_bynode': 0.7295049567365972, 'subsample': 0.8074190410592694, 'min_child_weight': 5, 'gamma'

Best trial: 186
Best value: 0.6702
Best params: {'max_depth': 4, 'learning_rate': 0.028623017835530524, 'n_estimators': 462, 'reg_alpha': 5.329370114774404, 'reg_lambda': 5.9557775079491995, 'colsample_bytree': 0.3220632706452828, 'colsample_bylevel': 0.7999898538362793, 'colsample_bynode': 0.8095801032633007, 'subsample': 0.8910668231396747, 'min_child_weight': 6, 'gamma': 0.8466930681580664, 'max_delta_step': 1, 'scale_pos_weight': 1.0125328886178924, 'grow_policy': 'lossguide', 'use_max_leaves': False}
Improvement over previous best: +0.03 percentage points
Model and study saved to best_xgb_model_optuna_enhanced.pkl

Final Model Accuracy: 0.6659
Current accuracy: 0.6659, target: ≥0.67
Gap to target: 0.41 percentage points

Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.71      0.69      2709
           1       0.67      0.62      0.64      2559

    accuracy                           0.67      5268
   macro avg       0.67

In [7]:
# Get feature importance as text
feature_importance = model.get_booster().get_score(importance_type='gain')
sorted_features = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)

print("Feature Importance (by weight):")
print("-" * 40)
for feature, importance in sorted_features:
    print(f"{feature:<25} {importance:>10}")

Feature Importance (by weight):
----------------------------------------
interaction_team_rolling_possession_touches_att_3rd_x_opp_team_rolling_conceded_possession_touches_att_3rd 167.0991668701172
is_home                   166.35202026367188
interaction_team_rolling_summary_sca_x_opp_team_rolling_conceded_summary_sca 135.79017639160156
interaction_team_rolling_summary_shots_x_opp_team_rolling_conceded_summary_shots 105.66584777832031
team_rolling_summary_shots 80.87718200683594
opp_team_rolling_conceded_summary_xg 76.79863739013672
opp_team_rolling_conceded_summary_shots 72.54606628417969
opp_team_rolling_conceded_possession_touches_att_pen_area 71.53253173828125
opp_team_rolling_conceded_summary_sca 70.7879867553711
team_rolling_summary_shots_on_target 69.68097686767578
team_rolling_summary_sca  63.26765441894531
team_rolling_possession_progressive_passes_received 60.18472671508789
team_rolling_possession_touches_live_ball 55.93585968017578
opp_team_rolling_conceded_possession_progre

In [8]:
test_predictions = model.predict(X_test_enhanced)
test_proba = model.predict_proba(X_test_enhanced)

test_df = df[["match_url", "match_date", "season", "division", 
                    "team", "opp_team", "is_home", "gw", "is_promoted?", "is_early_season?",
                    "shots"]].loc[X_test_enhanced.index].copy()

test_df['predicted_is_won'] = test_predictions  # Direct prediction (0 or 1)
test_df['prob_win'] = test_proba[:, 1]  # Just use column 1 (probability of class 1)

test_df["outcome"] = "under"
test_df["line"] = 11.5

test_df.to_csv("shots_test_predictions.csv", index=False)

test_df

,match_url,match_date,season,division,team,opp_team,is_home,gw,is_promoted?,is_early_season?,shots,predicted_is_won,prob_win,outcome,line
0,https://fbref.com/en/matches/e4bb1c35/Tottenha...,2025-05-25,2024-2025,Premier League,Tottenham,Brighton,1,38,0,0,3.0,0,0.422426,under,11.5
1,https://fbref.com/en/matches/e4bb1c35/Tottenha...,2025-05-25,2024-2025,Premier League,Brighton,Tottenham,0,38,0,0,22.0,0,0.438367,under,11.5
2,https://fbref.com/en/matches/1ff370e8/Bournemo...,2025-05-25,2024-2025,Premier League,Bournemouth,Leicester City,1,38,0,0,20.0,0,0.124457,under,11.5
3,https://fbref.com/en/matches/1ff370e8/Bournemo...,2025-05-25,2024-2025,Premier League,Leicester City,Bournemouth,0,38,1,0,3.0,1,0.729857,under,11.5
4,https://fbref.com/en/matches/36844e73/Newcastl...,2025-05-25,2024-2025,Premier League,Newcastle Utd,Everton,1,38,0,0,17.0,0,0.327117,under,11.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38076,https://fbref.com/en/matches/06aa495e/Reims-Li...,2024-08-17,2024-2025,Ligue 1,Lille,Reims,0,1,0,1,22.0,0,0.411223,under,11.5
38077,https://fbref.com/en/matches/35c1995a/Brest-Ma...,2024-08-17,2024-2025,Ligue 1,Brest,Marseille,1,1,0,1,9.0,0,0.330553,under,11.5
38078,https://fbref.com/en/matches/35c1995a/Brest-Ma...,2024-08-17,2024-2025,Ligue 1,Marseille,Brest,0,1,0,1,5.0,1,0.602742,under,11.5
38079,https://fbref.com/en/matches/a6b98e33/Le-Havre...,2024-08-16,2024-2025,Ligue 1,Le Havre,Paris S-G,1,1,0,1,5.0,1,0.521497,under,11.5
